In [ ]:
import nltk
import glob
import pandas as pd
import unicodedata

def norm(string):
    try:
        return unicodedata.normalize('NFKD', string).encode('ASCII', 'ignore').decode("latin-1")
    except:
        return string

def format_text(df, cols_names=None, stopwords=None):

    # format
    for ele in cols_names:
        df[ele] = df[ele].str.title().str.strip()
        for ene in stopwords:
            df[ele] = df[ele].str.replace(' ' + ene.title() + ' ', ' ' + ene + ' ')

    return df

data = glob.glob('*.xlsx')

# Intitution - campus dimension

In [ ]:
df = pd.DataFrame()
temp = pd.DataFrame()

for ele in data:
    print('Current file:', ele)
    temp = pd.read_excel(ele, header=1)
    temp.rename(columns={'NOMBRE INSTITUCIÓN': 'institution_id',
                         'NOMBRE INSTITUCIÓN ANUIES': 'institution_id',
                         'NOMBRE ESCUELA/CAMPUS/PLANTEL': 'campus_id',
                         'NOMBRE ESCUELA/CAMPUS ANUIES': 'campus_id'}, inplace=True)
    temp = temp[['institution_id', 'campus_id']].copy()
    for col in ['institution_id', 'campus_id']:
        temp[col] = temp[col].ffill()
    
    temp['backup_institution'] = temp['institution_id']
    temp['backup_campus'] = temp['campus_id']
    temp.drop_duplicates(subset=['institution_id', 'campus_id'], inplace=True)
    temp['version'] = ele
    temp.fillna(0, inplace=True)
    for col in ['backup_institution', 'backup_campus']:
        temp[col] = temp[col].astype(str).str.strip()
    temp['backup'] = (temp['backup_institution'] + temp['backup_campus']).str.strip()
    temp = temp[['institution_id', 'campus_id', 'backup', 'version']].copy()
    df = df.append(temp, sort=False)

version_replace = {
    'Anuario_Educacion_Superior_2019-2020_temp.xlsx': 2020,
    'anuies_licenciatura_2016-2017.xlsx': 2017,
    'anuies_licenciatura_2017-2018.xlsx': 2018,
    'anuies_licenciatura_2018-2019.xlsx': 2019,
    'anuies_posgrado_2016-2017.xlsx': 2017,
    'anuies_posgrado_2017-2018.xlsx': 2018,
    'anuies_posgrado_2018-2019.xlsx': 2019
}

df['version'].replace(version_replace, inplace=True)

In [ ]:
# t = df.copy()
df = t.copy()

# Raw -> Processed

In [ ]:
df = df.loc[~df['backup'].str.upper().str.contains('TOTAL')].copy()

df.drop_duplicates(subset=['backup'], inplace=True)

# text processing
for col in ['institution_id', 'campus_id']:
    
    # original names
    df['{}_name'.format(col.split('_')[0])] = df[col]

    df[col] = df[col].str.strip()

    df[col] = df[col].apply(lambda x: norm(x))

    exceptions = ['"']
    for e in exceptions:
        df[col] = df[col].str.replace(e, '')

    df[col] = df[col].str.replace('  ', ' ')
    df[col] = df[col].str.replace(' - ', ' ')
    df[col] = df[col].str.replace(', ', ' ')
    
    df[col] = df[col].str.replace(' (Cehem)', ' ')
    df[col] = df[col].str.replace('Esc. ', 'Escuela ')
    df[col] = df[col].str.replace('a C', 'A.C.')
    
    df[col] = df[col].str.strip()

institutions_exceptions = {
    'Colegio Decroly Plantel Puerto Vallarta': 'Colegio Decroly',
    'Escuela Juridica y Forense del Sureste Plantel Pachuca': 'Escuela Juridica y Forense del Sureste',
    'Escuela Normal de Licenciatura en Educacion Fisica de Calkini, Campeche': 'Escuela Normal de Licenciatura en Educacion Fisica de Calkini'
}

df['institution_id'].replace(institutions_exceptions, inplace=True)

# stopwords es
nltk.download('stopwords')
df = format_text(df, ['institution_id', 'campus_id'], stopwords=nltk.corpus.stopwords.words('spanish'))

df.shape, df.drop_duplicates(subset=['institution_id', 'campus_id']).shape

df['backup_raw'] = (df['institution_id'] + df['campus_id']).str.strip()

df.to_csv('raw_to_institution.csv', index=False)

In [ ]:
df.shape, df.drop_duplicates(subset=['institution_id', 'campus_id']).shape

In [ ]:
df.drop_duplicates(subset=['institution_id', 'campus_id'], inplace=True)

In [ ]:
test = df.loc[(df['institution_id'].duplicated()) & (~df['institution_name'].duplicated())].shape[0]
if test > 0:
    # duplicate ids with different names
    duplicated_names = list(df.loc[(df['institution_id'].duplicated()) & (~df['institution_name'].duplicated()), 'institution_id'].unique())

    for ele in duplicated_names:

        # get latest version of name
        latest_name_version = df.loc[df['institution_id'] == ele, 'version'].max()

        # set latest institution name to all ids
        df.loc[df['institution_id'] == ele, 'institution_name'] = \
            list(df.loc[(df['institution_id'] == ele) & (df['version'] == latest_name_version), 'institution_name'])[0]
    print('Duplicate ids fix!')
else:
    print('Clean ids!')

# Processed -> id

In [ ]:
# campus id
df['count'] = 1
count = 0
while df.loc[df.duplicated(subset=['institution_id', 'count'], keep='first')].shape[0] > 0:
    df.loc[df.duplicated(subset=['institution_id', 'count'], keep='first'), 'count'] = \
        df.loc[df.duplicated(subset=['institution_id', 'count'], keep='first'), 'count'] + 1
    count += 1
print('Campus Steps Count:', count)

# max_group = str(df['count'].max())
# len of institutions = 4
max_group = range(5)
df['count'] = df['count'].astype(str).str.zfill(len(max_group))

institution_id = df.drop_duplicates(subset=['institution_id']).copy()

institution_id['id'] = range(1, institution_id.shape[0]+1)

df['institution_id_final'] = df['institution_id'].replace(dict(zip(institution_id['institution_id'], institution_id['id'])))

df['campus_id'] = (df['institution_id_final'].astype(str) + df['count'].astype(str)).astype(int)
df['institution_id'] = df['institution_id_final']
df.drop(columns=['version', 'count', 'institution_id_final'], inplace=True)

for col in ['institution_name', 'campus_name']:
    df[col] = df[col].str.title()

df.to_csv('dim_institutions.csv', index=False)